In [14]:
from collections import deque
import pandas as pd
from IPython.display import display


# Set pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Define the time quantum
time_quantum = 1

# Process data: ProcessID, ArrivalTime, BurstTime
#processes = [("A", 4, 4), ("B", 3, 3), ("C", 0, 2), ("D", 2, 7), ("E", 5, 2), ("F", 1, 4)] # exercise
#processes = [("A", 4, 4), ("B", 3, 3), ("C", 0, 2), ("D", 2, 7), ("E", 5, 2), ("F", 1, 4)] #test
processes = [("A", 4, 3), ("B", 3, 7), ("C", 1, 4), ("D", 5, 2), ("E", 0, 2), ("F", 2, 4)]  # example
# Sort processes by their arrival time
processes.sort(key=lambda x: x[1])

# Initialize variables
queue = deque()
current_time = 0
timeline = []
waiting_time = {process[0]: 0 for process in processes}
turnaround_time = {process[0]: 0 for process in processes}
remaining_burst_time = {process[0]: process[2] for process in processes}

# Start the round-robin scheduling
while processes or queue:
    # Add processes to the queue that have arrived
    while processes and processes[0][1] <= current_time:
        queue.append(processes.pop(0))

    if queue:
        current_process = queue.popleft()
        process_id, _, burst_time = current_process

        # Calculate execution time (either full quantum or remaining burst time)
        exec_time = min(time_quantum, remaining_burst_time[process_id])
        remaining_burst_time[process_id] -= exec_time

        # Record the timeline of the process execution
        timeline.append((process_id, current_time, current_time + exec_time))
        current_time += exec_time

        # If the process is not finished, put it back in the queue
        if remaining_burst_time[process_id] > 0:
            queue.append(current_process)
        else:
            # Process is finished, calculate turnaround time
            turnaround_time[process_id] = current_time - current_process[1]

        # Update waiting times for processes in the queue
        for proc in queue:
            waiting_time[proc[0]] += exec_time

    else:
        # No process in the queue, move forward in time
        current_time = processes[0][1] if processes else current_time + 1

# Calculate average waiting and turnaround times
average_waiting_time = sum(waiting_time.values()) / len(waiting_time)
average_turnaround_time = sum(turnaround_time.values()) / len(turnaround_time)

# Create a DataFrame for better visualization
df_timeline = pd.DataFrame(timeline, columns=["ProcessID", "StartTime", "EndTime"])
df_waiting_turnaround = pd.DataFrame(list(waiting_time.items()), columns=["ProcessID", "WaitingTime"])
df_waiting_turnaround["TurnaroundTime"] = df_waiting_turnaround["ProcessID"].map(turnaround_time)

# Creating a table for the Round Robin execution timeline

# Define the maximum time units for the table
max_time_units = 30

# Initialize the table with empty strings
execution_table = {process_id: [" " for _ in range(max_time_units)] for process_id in waiting_time}

# Fill the table with 'X' where processes are executing
for process_id, start, end in timeline:
    for time_unit in range(start, min(end, max_time_units)):
        execution_table[process_id][time_unit] = "X"

# Convert the table to a DataFrame for better visualization
df_execution_table = pd.DataFrame(execution_table)
df_execution_table.index += 1  # Adjusting index to start from 1
df_execution_table.index.name = "_"
df_execution_table.columns.name = "ProcessID"

# Transpose the DataFrame to swap rows and columns
df_execution_transposed = df_execution_table.transpose()

# Reordering the columns to display processes in the order A, B, C, D, E, F
ordered_processes = ['A', 'B', 'C', 'D', 'E', 'F']
df_execution_ordered = df_execution_transposed.loc[ordered_processes]

# Styling and displaying the DataFrame directly
df_execution_ordered.style.map(
    lambda x: 'background-color: lightgreen' if x == 'X' else 'background-color: white'
)
display(df_execution_ordered)


_,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
ProcessID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A,,,,,,,,X,,,,,X,,,,,X,,,,,,,,,,,,
B,,,,,,X,,,,,X,,,,,X,,,X,X,X,X,,,,,,,,
C,,,X,,X,,,,X,,,,,X,,,,,,,,,,,,,,,,
D,,,,,,,,,,X,,,,,X,,,,,,,,,,,,,,,
E,X,X,,,,,,,,,,,,,,,,,,,,,,,,,,,,
F,,,,X,,,X,,,,,X,,,,,X,,,,,,,,,,,,,
